<a href="https://colab.research.google.com/github/SanjaPanda/car-plate-detector/blob/main/Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing important libraries

import glob
import matplotlib.pyplot as plt
import cv2

import shutil
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
#Change directory to working directory

%cd '/content/drive/MyDrive/car-plate-detector/workspace/training_demo/'

/content/drive/MyDrive/car-plate-detector/workspace/training_demo


# Data Wrangling

### Step 1: Images are split into 2 folders 'Train' and 'Test'.
### Step 2: XML to CSV

      Extracting data from XML to CSV. These sheets would be later used to create TFRecords

In [ ]:
def xml_to_csv(path):
    xml_list = [] 
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']

    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        filename = root.find('filename').text
        height = root.find('size').find('height').text
        width = root.find('size').find('width').text
        for obj in root.findall('object'):
            clas = obj.find('name').text
            xmin = obj.find('bndbox').find('xmin').text
            xmax = obj.find('bndbox').find('xmax').text
            ymin = obj.find('bndbox').find('ymin').text
            ymax = obj.find('bndbox').find('ymax').text
            xml_list.append([filename,width,height,clas,xmin,ymin,xmax,ymax])
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [ ]:
xml_df_train = xml_to_csv('images/train')
print(xml_df_train.head())

xml_df_test = xml_to_csv('images/test')
print(xml_df_test.head())

     filename width height    class xmin ymin xmax ymax
0  Cars94.png   400    319  licence  176  210  251  248
1  Cars92.png   600    400  licence  281  185  393  271
2   Cars6.png   500    375  licence  112  114  373  239
3  Cars71.png   400    225  licence  254  144  284  152
4  Cars71.png   400    225  licence   86  150  125  161
      filename width height    class xmin ymin xmax ymax
0  Cars415.png   400    300  licence  133  157  261  192
1  Cars383.png   400    267  licence   54  216   88  228
2  Cars386.png   600    450  licence   83  307  221  368
3  Cars417.png   400    295  licence   90  145  273  196
4  Cars403.png   400    320  licence  296  189  332  209


In [ ]:
xml_df_train.to_csv('annotations/carplate_labels_train.csv', index=None)
xml_df_test.to_csv('annotations/carplate_labels_test.csv', index=None)

### Step 3: Prepare for transfer learning for your model
  * Objective is to pick attributes from pre-trained model to create model.
  *  Create a copy of 'pipeline.config' and later update it as per requirements.


In [ ]:
#Unpack model
shutil.unpack_archive('pre-trained-models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz', 'pre-trained-models/')

#Create a copy of 'pipeline.config'
shutil.copy('pre-trained-models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config', 'models/my_ssd_mobilenet_v2_fpnlite_640x640/', follow_symlinks = True)

'models/my_ssd_mobilenet_v2_fpnlite_640x640/pipeline.config'

## Update the pipeline.config file

Other Parametes:

    number of classes: 1

    batch_size: 16

update checkpoint details:

    fine_tune_checkpoint ssd: "/content/drive/MyDrive/car-plate-detector/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0"

    type: detection

    no. of steps: 10000

Paths:

    label_map_path: /content/drive/MyDrive/car-plate-detector/workspace/training_demo/annotations/label_map.pbtxt

    train_input_path: /content/drive/MyDrive/car-plate-detector/workspace/training_demo/annotations/train.record

    test_input_path: /content/drive/MyDrive/car-plate-detector/workspace/training_demo/annotations/test.record